In [13]:
from ctrace import *
import pandas as pd
import numpy as np

In [14]:
# Load in results file
with open(PROJECT_ROOT /"output"/"run_6Mm2w/" / "results.csv", "r") as csv:
    df = pd.read_csv(csv)
print(len(df))
df.head()

11200


,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap
0,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,1217,6233,11135,16931,28,88.819097,114.2,0.601694,661.962407,53
1,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,1217,6233,11135,16931,28,61.833362,85.2,0.921509,661.962407,405
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1217,6233,11135,16931,28,60.874122,77.4,1.060092,661.962407,426
3,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,1217,6233,11135,16931,28,63.834657,81.0,0.322041,661.962407,-1
4,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,2359,25664,32967,145552,59,523.116239,1423.6,3.947826,3733.643370,15


In [15]:
# Extract the Gurobi MILP optimal runs
optimal = df[df["method"] == "mip_gurobi"]
print(len(optimal))
optimal.head()

2800


,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1217,6233,11135,16931,28,60.874122,77.4,1.060092,661.962407,426
11,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1584,7527,12026,19278,27,74.677546,101.4,1.439489,846.461136,412
25,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,5772,26012,16285,42903,31,242.382803,465.8,7.878851,4839.647687,325
28,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,4854,31285,22040,82500,32,361.245810,875.4,14.611467,5702.689558,287
33,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,5736,26662,16890,47422,27,261.335158,515.2,12.540073,5014.496636,289


In [16]:
# Remove duplicates from different trials
optimal = optimal[optimal["trial_id"] == 0]

# Only these 4 ("G", "p", "budget", "from_cache") are needed to uniquely identify simulation parameters
optimal = optimal[["G", "p", "budget", "from_cache", "mip_value"]]
optimal = optimal.rename(columns={"mip_value": "optimal_mip_value"})

print(len(optimal))
optimal.head()

2800


,G,p,budget,from_cache,optimal_mip_value
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,60.874122
11,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,74.677546
25,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,242.382803
28,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,361.245810
33,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,261.335158


In [17]:
# Merge on the aforementioned key columns
df = df.merge(optimal, on=["G", "p", "budget", "from_cache"])

In [18]:
df.head()


,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value
0,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,1217,6233,11135,16931,28,88.819097,114.2,0.601694,661.962407,53,60.874122
1,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,1217,6233,11135,16931,28,61.833362,85.2,0.921509,661.962407,405,60.874122
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1217,6233,11135,16931,28,60.874122,77.4,1.060092,661.962407,426,60.874122
3,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,1217,6233,11135,16931,28,63.834657,81.0,0.322041,661.962407,-1,60.874122
4,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,2359,25664,32967,145552,59,523.116239,1423.6,3.947826,3733.643370,15,432.438811


In [19]:
df["mip_ratio_gap"] = df["mip_value"] / df["optimal_mip_value"]
df.head()

,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value,mip_ratio_gap
0,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,1217,6233,11135,16931,28,88.819097,114.2,0.601694,661.962407,53,60.874122,1.459062
1,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,1217,6233,11135,16931,28,61.833362,85.2,0.921509,661.962407,405,60.874122,1.015758
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1217,6233,11135,16931,28,60.874122,77.4,1.060092,661.962407,426,60.874122,1.000000
3,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,1217,6233,11135,16931,28,63.834657,81.0,0.322041,661.962407,-1,60.874122,1.048634
4,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,2359,25664,32967,145552,59,523.116239,1423.6,3.947826,3733.643370,15,432.438811,1.209688


In [20]:
# Check that mip_ratio_gap >= 1 for all elements
assert len(df[df["mip_ratio_gap"] < 1]) == 0

In [21]:
df = df.sort_values(by=['v2_size'])

In [22]:
dfk = [df[df["budget"] == x] for x in range(600, 1201, 100)]
dfk[0].head()

,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value,mip_ratio_gap
1356,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,538,2886,6427,8390,25,40.197954,50.6,0.308147,278.972063,123,21.503628,1.869357
1357,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,538,2886,6427,8390,25,21.619965,27.2,0.779478,278.972063,489,21.503628,1.005410
1358,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,538,2886,6427,8390,25,22.388281,33.0,0.017547,278.972063,-1,21.503628,1.041140
1359,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,538,2886,6427,8390,25,21.503628,26.2,0.734187,278.972063,500,21.503628,1.000000
758,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,529,2808,6472,8273,23,21.620672,26.0,0.818339,276.134672,509,21.620672,1.000000


In [23]:
# Create an column that splits dataset into n groups
num_splits = 4
col = []
for i in range(num_splits):
    col += [i] * int(len(df) / num_splits)
df["v2_id"] = col

In [33]:
df.head()

,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value,mip_ratio_gap,v2_id
6159,montgomery,0.078,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,538,2886,6427,8390,25,14.974574,18.0,0.483384,278.972063,767,14.893481,1.005445,0
9357,montgomery,0.078,1100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,538,2886,6427,8390,25,11.312337,13.8,0.431276,278.972063,978,11.271384,1.003633,0
9356,montgomery,0.078,1100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,538,2886,6427,8390,25,32.454532,40.4,0.718345,278.972063,414,11.271384,2.879374,0
6158,montgomery,0.078,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,538,2886,6427,8390,25,14.893481,16.8,0.402228,278.972063,792,14.893481,1.000000,0
6157,montgomery,0.078,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,538,2886,6427,8390,25,35.761233,45.6,0.313535,278.972063,285,14.893481,2.401133,0


In [34]:
df.to_csv("results_appended.csv")

mean                                                     \
                 duration                                  min_exposed_value   
v2_id                   0          1          2          3                 0   
method                                                                         
dependent        1.982543   4.767330  11.358494  20.199614         92.484857   
greedy_weighted  0.164701   0.455687   0.743440   0.541064         85.662000   
mip_gurobi       2.667303  10.459121  27.821475  41.904462         90.039143   
random           1.410377   3.214172   5.400360   5.985428        132.216571   

                                                                               \
                                                      mip_ratio_gap             
v2_id                     1            2            3             0         1   
method                                                                          
dependent        361.444571   918.401143   964.822286      1.024761  1.044089   
greedy_weighted  336.194286   855.168000   894.599429      1.059721  1.060155   
mip_gurobi       352.641714   895.541143   928.693143      1.000000  1.000000   
random           436.345143  1045.656000  1142.120571      1.694375  1.314827   

                 ...        max                                               \
                 ...   duration             min_exposed_value                  
v2_id            ...          2           3                 0      1       2   
method           ...                                                           
dependent        ...  24.713740   60.053831             238.4  619.2  1265.4   
greedy_weighted  ...   2.302402    2.154264             225.0  599.4  1216.6   
mip_gurobi       ...  90.494773  103.989422             243.0  608.6  1249.0   
random           ...  11.313290   14.601623             297.2  698.8  1406.0   

                                                                     
                        mip_ratio_gap                                
v2_id                 3             0         1         2         3  
method                                                               
dependent        1314.2      1.061937  1.063669  1.082586  1.131799  
greedy_weighted  1238.0      1.093158  1.086814  1.087524  1.128314  
mip_gurobi       1300.8      1.000000  1.000000  1.000000  1.000000  
random           1507.0      3.142609  1.526970  1.332190  1.750537  

[4 rows x 24 columns]